In [26]:
def split(input_str, delimiter, max_splits=50):
    # Convert input to string if it's bytes
    if isinstance(input_str, bytes):
        input_str = input_str.decode('utf-8')
        
    items = input_str.split(delimiter)
    # Limit the number of splits to max_splits or 50, whichever is smaller
    limited_items = items[:min(max_splits, 50)]
    
    return limited_items

def isNeglectable(pos):
    neglectable_set = {'CC', 'CD', 'DT', 'EX', 'IN', 'LS', 'MD', 'PDT', 'POS', 'SYM', 'TO', 'UH', 'VBZ', "','", '', '.'}
    return pos in neglectable_set

def trimQuotes(str_):
    if len(str_) == 0:
        return ""
    return str_[1:-1]

def trimPr(str_):
    if len(str_) == 0:
        return ""
    return str_[2:-1]

In [48]:
from typing import List, NamedTuple


class Token:
    def __init__(self, str_, pos, lemma, ner):
        self.str = str_
        self.pos = pos
        self.lemma = lemma
        self.ner = ner


class Parser:
    def __init__(self):
        self.tokens = []

    @staticmethod
    def parse(st):
        tokens = Parser.getTokens(st)
        for i, token in enumerate(tokens):
            pos_trimmed = Parser.trimQuotes(token.pos)
            print(f"{pos_trimmed}  ----- {i}")
            if Parser.isNeglectable(pos_trimmed):
                continue
            print(f"{token.str}   {token.pos}")

    @staticmethod
    def get_tokens(st: str) -> List[Token]:
        i = 0
        toks = []
        
        # Find the first '['
        while st[i] != '[':
            i += 1
        i += 1

        while i < len(st) and st[i] != ']':
            t = i
            
            # Find the next ']'
            while i < len(st) and st[i] != ']':
                i += 1
            token_str = st[t:i]
            print(f"Token string: {token_str}")
            list_ = split(token_str, ',', 20)
            print(f"List: {list_}")

            sublist = split(list_[0], ':', 4)
            str_ = sublist[1] if len(sublist) > 1 else ""
            
            sublist = split(list_[1], ':', 4)
            pos = sublist[1] if len(sublist) > 1 else ""
            
            sublist = split(list_[2], ':', 4)
            lemma = sublist[1] if len(sublist) > 1 else ""
            
            sublist = split(list_[3], ':', 4)
            ner = sublist[1] if len(sublist) > 1 else ""
            
            tok = Token(str_, pos, lemma, ner)
            toks.append(tok)
            
            i += 1

        return toks

In [49]:

def process(s: str) -> str:
    p = Parser()
    toks = p.get_tokens(s)
    st = []

    for i, tok in enumerate(toks):
        pos_trimmed = trimQuotes(tok.pos)
        print(f"{pos_trimmed}  ----- {i}")
        if isNeglectable(pos_trimmed):
            continue
        print(f"{tok.str} ()()()()()()(  {tok.pos}")

        if tok.str and tok.str[0] == ' ':
            st.append(trimPr(tok.str))
        else:
            st.append(tok.str)

        st.append(" ")
        st.append(trimQuotes(tok.pos))
        st.append(" ")

    return "".join(st)

In [50]:
# # Test the function
# test_str = "[str: 'A', pos: 'DT', lemma: 'a', ner: 'O'], [str: 'hill', pos: 'NN', lemma: 'hill', ner: 'O']"
# result = process(test_str)
# print(result)
# Test the function
result = process("merge(drs([[1001]:_G32965], [[1002]:pred(_G32965, hill, n, 0)]), merge(merge(drs([[1004]:_G33027, [1007]:_G33048], [[1005]:pred(_G33027, area, n, 0), [1007]:pred(_G33048, land, n, 0)]), merge(merge(drs([[1010]:_G33133], [[1010]:prop(_G33133, drs([], [[1010]:pred(_G33048, higher, a, 0)]))]), alfa(def, drs([[1012]:_G33217], [[1013]:pred(_G33217, land, n, 0)]), drs([[1016]:_G33267, [1015]:_G33288], [[1016]:pred(_G33267, it, n, 0), [1015]:pred(_G33288, surround, v, 0), []:pred(_G33288, event, n, 1), []:pred(_G33133, event, n, 1), [1015]:rel(_G33288, _G33217, agent, 0), [1015]:rel(_G33288, _G33267, patient, 0), [1011]:rel(_G33133, _G33217, than, 0)]))), drs([], [[1006]:rel(_G33027, _G33048, of, 0)]))), drs([[1003]:_G33517], [[]:pred(_G33517, event, n, 1), [1003]:prop(_G33517, drs([], [[1003]:eq(_G32965, _G33027)]))]))) ).")
print(result)

Token string: [1001
List: ['[1001']


IndexError: list index out of range